In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
# !tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
# !pip install -q findspark

# Set Environment Variables
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
os.environ["SPARK_HOME"] = r'C:\spark-3.1.1-bin-hadoop3.2'
os.environ["HADOOP_HOME"] = r'C:\hadoop'

# Start a SparkSession
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = r'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [6]:
# Create the vine_table DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [7]:
# Filter data to create a DataFrame with 20 or more total votes
cleaned_vine_df = vine_df.filter("total_votes >= 20")
cleaned_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|ROP6ITXO8K5V2 |5          |23           |27         |N   |Y                |
|R3ND1LVU7AXCVF|1          |21           |21         |N   |Y                |
|R9I5FOLKU99RY |5          |19           |20         |Y   |N                |
|R1QS8AOD6HX3ED|4          |59           |81         |N   |N                |
|R3ED60RC69CCQ6|5          |22           |23         |Y   |N                |
|R2MDI1TP46VSYD|3          |9            |21         |N   |Y                |
|R2JM687C525WR9|3          |33           |33         |Y   |N                |
|R2LWX4TZ67FWPT|4          |50           |50         |N   |N                |
|R29IYHPYD14AGI|3          |84           |84         |N   |Y                |
|R1F2I723WRK5QV|5          |20           |20         |N   |Y    

In [8]:
# Filter data to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50%
helpful_vine_df = cleaned_vine_df.filter("helpful_votes / total_votes >=0.5")
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|     

In [9]:
# Create a DataFrame or table where there is a Vine review 
paid_vine_df = helpful_vine_df.filter("vine == 'Y'")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|
|R2H4VEFWHP50M3|          5|           26|         36|   Y|                N|
|R3DP6Y0A8WXUOV|          5|           21|         21|   Y|                N|
| R49L85GZIKH4D|          3|           33|         36|   Y|     

In [10]:
paid_vine_df = helpful_vine_df.filter(helpful_vine_df["vine"] == "Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|
|R2H4VEFWHP50M3|          5|           26|         36|   Y|                N|
|R3DP6Y0A8WXUOV|          5|           21|         21|   Y|                N|
| R49L85GZIKH4D|          3|           33|         36|   Y|     

In [11]:
#Create a DataFrame or table where there is a non-Vine review 
unpaid_vine_df = helpful_vine_df.filter(helpful_vine_df["vine"] == "N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|
|R30PJ9XUHFTY5D|          5|           42|         43|   N|                N|
|R2F7D9N3SSQC20|          4|           32|         34|   N|     

In [12]:
# The total number of reviews
total_number_reviews = helpful_vine_df.count()
total_number_reviews

63294

In [13]:
# The total number of paid reviews
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

1266

In [14]:
# The total number of unpaid reviews
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

62028

In [15]:
# The number of 5-star reviews
Number_of_paid_5_star_reviews = paid_vine_df.filter(paid_vine_df['star_rating'] == 5)
Number_of_paid_5_star_reviews.count()

432

In [16]:
# The percentage of paid 5-star reviews
paid_percentage = (Number_of_paid_5_star_reviews.count()/helpful_vine_df.count()) *100
paid_percentage

0.6825291496824344

In [17]:
# The number of unpaid 5-star reviews
Number_of_unpaid_5_star_reviews = unpaid_vine_df.filter(unpaid_vine_df['star_rating'] == 5)
Number_of_unpaid_5_star_reviews.count()

29982

In [18]:
# The percentage of unpaid 5-star reviews
unpaid_percentage = (Number_of_unpaid_5_star_reviews.count()/helpful_vine_df.count()) *100
unpaid_percentage

47.36941890226562